## 322. Coin Change
- Description:
  <blockquote>
    You are given an integer array `coins` representing coins of different denominations and an integer `amount` representing a total amount of money.

    Return _the fewest number of coins that you need to make up that amount_. If that amount of money cannot be made up by any combination of the coins, return `-1`.

    You may assume that you have an infinite number of each kind of coin.

    **Example 1:**

    ```
    Input: coins = [1,2,5], amount = 11
    Output: 3
    Explanation: 11 = 5 + 5 + 1

    ```

    **Example 2:**

    ```
    Input: coins = [2], amount = 3
    Output: -1

    ```

    **Example 3:**

    ```
    Input: coins = [1], amount = 0
    Output: 0

    ```

    **Constraints:**

    -   `1 <= coins.length <= 12`
    -   `1 <= coins[i] <= 2<sup>31</sup> - 1`
    -   `0 <= amount <= 10<sup>4</sup>`
  </blockquote>

- URL: [Problem_URL](https://leetcode.com/problems/coin-change/description/)

- Topics: Recursion+memo, DP, BFS

- Difficulty: Medium 

- Resources: example_resource_URL

### Solution 1, Recursion with Memoization
Solution description
- Time Complexity: O(S * N)
  - where S is the amount, n is denomination count. In the worst case the recursive tree of the algorithm has height of S and the algorithm solves only S subproblems because it caches precalculated solutions in a table.
  - Each subproblem is computed with n iterations, one by coin denomination. Therefore there is O(S∗n) time complexity.
- Space Complexity: O(S)
  - where S is the amount to change. We use extra space for the memoization table.

In [ ]:
class Solution:
    def coinChange(self, coins: List[int], amount: int) -> int:
        def dfs(rem: int) -> int:
            if rem in memo:
                return memo[rem]
            
            if rem < 0:
                return -1
            
            if rem == 0:
                return 0

            min_cost = float('inf')
            for coin in coins:
                res = dfs(rem - coin)
                if res != -1:
                    min_cost = min(min_cost, res + 1)

            memo[rem] = -1 if min_cost == float('inf') else min_cost
            return memo[rem]

        memo = {}  # rem -> min coins (or -1 if impossible)
        return dfs(amount)

### Solution 2, Dynamic programming - Bottom up

Input: coins = [1,2,3], amount = 6
Output: 2
Explanation: 6 = 3 + 3

```Coin = 1, x = [1,2,3,4,5,6]
dp[1] = min(dp[1], dp[0]+1) = 1
dp[2] = min(dp[2], dp[1]+1) = 2
dp[3] = min(dp[3], dp[2]+1) = 3
dp[4] = min(dp[4], dp[3]+1) = 4
dp[5] = min(dp[5], dp[4]+1) = 5
dp[6] = min(dp[6], dp[5]+1) = 6


Coin = 2, x = [2,3,4,5,6]
dp[2] = min(dp[2], dp[0]+1) = 1
dp[3] = min(dp[3], dp[1]+1) = 2
dp[4] = min(dp[4], dp[2]+1) = 2
dp[5] = min(dp[5], dp[3]+1) = 3
dp[6] = min(dp[6], dp[4]+1) = 3


Coin = 3, x = [3,4,5,6]
dp[3] = min(dp[3], dp[0]+1) = 1
dp[4] = min(dp[4], dp[1]+1) = 2
dp[5] = min(dp[5], dp[2]+1) = 2
dp[6] = min(dp[6], dp[3]+1) = 2
```

for each amount the min number of coins is the overall minimmum of the min number of coins needed for amt-c1, amt-c2, amtc3 ....
for example dp[6] = min(dp[5], dp[4], dp[3]) if there were 3 coins [1,2,3] and amount needed was 6

For the iterative solution, we think in bottom-up manner. Before calculating F(i), we have to compute all minimum counts for amounts up to i. On each iteration i of the algorithm F(i) is computed as min⁡j=0…n−1 F(i−cj)+1

- Time Complexity: O(S * N)
  - On each step the algorithm finds the next F(i) in n iterations, where 1 ≤ i ≤ S. Therefore in total the iterations are S∗n.
- Space Complexity: O(S)
  - where S is the amount to change. We use extra space for the memoization table.

In [ ]:
class Solution:
    def coinChange(self, coins: List[int], amount: int) -> int:
        dp = [float('inf')] * (amount + 1)
        dp[0] = 0
        
        for coin in coins:
            for amt in range(coin, amount + 1):
                dp[amt] = min(dp[amt], dp[amt - coin] + 1)
                
        return dp[amount] if dp[amount] != float('inf') else -1 

### Solution 3, Optimum Iterative BFS

find the shortet path from 0 to required amount.  
unweighted graph where nodes are sums and edges add a coin. BFS finds shortest path (fewest edges), which maps to fewest coins.  
BFS visits sums in order of increasing coin count. The first time amount is popped/seen corresponds to the smallest number of coins.

- Time Complexity: O(S * N)
  - in worst case (each sum 0..amount visited once, each with up to len(coins) neighbors).
- Space Complexity: O(S)
  - for visited and queue.

In [ ]:
class Solution:
    def coinChange(self, coins: List[int], amount: int) -> int:
        if not amount:  # Don't need any coin.
            return 0
        
        # totalCoins (aka min coins for current amount), current amount
        queue = deque([(0, 0)])
        visited = [True] + [False] * amount
        
        while queue:
            totalCoins, currVal = queue.popleft()
            totalCoins += 1  # Take a new coin.
            
            for coin in coins:
                nextVal = currVal + coin
                if nextVal == amount:
                    # Because BFS explores by increasing coin count, this is guaranteed minimal
                    return totalCoins

                # If nextVal > amount it is pruned (no need to explore).
                if nextVal < amount:  # Could add more coins.
                    if not visited[nextVal]:  # Current value not checked.
                        visited[nextVal] = True  # Prevent checking again. prevents revisiting same sum via longer paths 
                        queue.append((totalCoins, nextVal))

        return -1  # if queue empties without reaching amount, return -1 as we cannot find any combination.